# TP1

Autores: Afonso Ferreira pg52669, Tiago Rodrigues pg52705

## Exercício 1

### Enunciado

Use a package Cryptography   e  o package ascon para  criar um comunicação privada assíncrona em modo  “Lightweight Cryptography” entre um agente Emitter e um agente Receiver que cubra os seguintes aspectos:

- a) Autenticação do criptograma e dos metadados (associated data) usando Ascon em modo de cifra.

- b) As chaves de cifra, autenticação  e  os “nounces” são gerados por um gerador pseudo aleatório (PRG)  usando o Ascon em modo XOF. As diferentes chaves para inicialização do PRG são inputs do emissor e do receptor.

- c) Para implementar a comunicação cliente-servidor use o package python asyncio

In [1]:
from ascon import encrypt, decrypt, hash
import asyncio
import os

Definição das funções, `emitter`, `receiver` e `pseudoRandomGenerator`.

* `emitter` - Criação da cifra atráves da função encrypt da package Ascon

* `receiver` - Decifrar a cifra recebida

* `pseudoRandomGenerator` - Criação da chave da cifra e nounce usando o Ascon em modo XOF

In [2]:
def emitter(plaintext, key, nonce, associateddata):
    cifra = encrypt(key, nonce, associateddata, plaintext, variant="Ascon-128")
    return cifra

def receiver(key, nonce, associateddata, cifra):
    receivedPlainText = decrypt(key, nonce, associateddata, cifra, variant="Ascon-128")
    if receivedPlainText == None: print("Verification failed :(")
    return receivedPlainText

def pseudoRandomGenerator():
    keySeed = os.urandom(16)
    key = hash(keySeed, variant="Ascon-Xof", hashlength=16)

    nonceSeed = os.urandom(16)
    nonce = hash(nonceSeed, variant="Ascon-Xof", hashlength=16)
    return key, nonce

Implementação da comunicação cliente-servidor

* `handle_echo` - comporta-se como o server, onde está à escuta por um cliente e recebe uma mensagem encriptada e decifra-a.

* `tcp_echo_client` - como o nome refere, trata-se do cliente, onde este manda uma mensagem encriptada para o servidor e recebe de volta a sua mensagem original.

* `main` - função para pôr ambos a correr

In [3]:
# Server
async def handle_echo(reader, writer):
    data = await reader.read(100)
    message = data

    addr = writer.get_extra_info('peername')

    print(f"Server has received {message} from client {addr}")

    print("Decrypting...")
    data = receiver(key, nonce, associateddata, message)

    print(f"Server has sent: {data}")
    writer.write(data)
    await writer.drain()

    print("Closing the server connection")
    writer.close()
    

async def main():
    server = await asyncio.start_server(
        handle_echo, '127.0.0.1', 8889)

    addr = server.sockets[0].getsockname()
    print(f'Serving on {addr}')

# Client
async def tcp_echo_client():
    await asyncio.sleep(3)
    reader, writer = await asyncio.open_connection(
        '127.0.0.1', 8889)

    print("Sending: ", plaintext)

    print('Encrypting...')
    cifra = emitter(plaintext, key, nonce, associateddata)

    print(f'Client has sent: {cifra}')
    writer.write(cifra)

    data = await reader.read(100)
    print(f'Client has received: {data}')

    print('Closing the client connection')
    writer.close()

async def run_client_and_server():
    await asyncio.gather(tcp_echo_client(), main())


plaintext = b"Anacleto manda mensagem a Bernardina"
key, nonce = pseudoRandomGenerator()
associateddata = b"ASCON"

await run_client_and_server()

Serving on ('127.0.0.1', 8889)
Sending:  b'Anacleto manda mensagem a Bernardina'
Encrypting...
Client has sent: b'\x88/\xcb(\xe9\xb5Ft\x03\x07\xe7\xe6\xe8\x86\x0b\xcc x8\x12\x1e\xe6+\x9a\xd5\x9c\x13\\\x93\xffm1p\x02h&\tN\xc5\xcb@jf2\x98yf\xa1\x92\xfc\x12H'
Server has received b'\x88/\xcb(\xe9\xb5Ft\x03\x07\xe7\xe6\xe8\x86\x0b\xcc x8\x12\x1e\xe6+\x9a\xd5\x9c\x13\\\x93\xffm1p\x02h&\tN\xc5\xcb@jf2\x98yf\xa1\x92\xfc\x12H' from client ('127.0.0.1', 53848)
Decrypting...
Server has sent: b'Anacleto manda mensagem a Bernardina'
Closing the server connection
Client has received: b'Anacleto manda mensagem a Bernardina'
Closing the client connection


## Exercício 2

### Enunciado

Use o “package” Cryptography para
- a) Implementar uma AEAD com “Tweakable Block Ciphers” conforme está descrito na última secção do texto +Capítulo 1: Primitivas Criptográficas Básicas.  A cifra por blocos primitiva, usada para gerar a “tweakable block cipher”, é o AES-256 ou o ChaCha20.
- b) Use esta cifra para construir um canal privado de informação assíncrona com acordo de chaves feito com “X448 key exchange” e “Ed448 Signing&Verification” para autenticação  dos agentes. Deve incluir uma fase de confirmação da chave acordada.

In [4]:
from cryptography.hazmat.primitives.asymmetric.x448 import X448PrivateKey
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives.ciphers.algorithms import AES256
from cryptography.hazmat.primitives.asymmetric.x448 import X448PrivateKey
from cryptography.hazmat.primitives.asymmetric.ed448 import Ed448PrivateKey
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.ciphers.aead import ChaCha20Poly1305
import asyncio
import os

A cifra por blocos primitiva que escolhemos foi a do ChaCha20.

Com isto foi necessário a definição das funções, `emitter`, `receiver` e `pseudoRandomGenerator`.

* `emitter` - Criação da cifra atráves da função encrypt importada da package ChaCha20Poly1305 que implementa a AEAD.

* `receiver` - Decifrar a cifra recebida.

* `pseudoRandomGenerator` - Criação da chave da cifra com um acordo de chaves feito com “X448 key exchange” e “Ed448 Signing&Verification” para autenticação.

In [5]:
def emitter(plaintext, key, nonce, associateddata):
    # Cifrar os dados
    cifra = chacha.encrypt(nonce, plaintext, associateddata)
    return cifra

def receiver(key, nonce, associateddata, cifra):
    # Decifrar os dados
    receivedPlainText = chacha.decrypt(nonce, cifra, associateddata)
    if receivedPlainText == None: print("Verification failed :(")
    return receivedPlainText

def pseudoRandomGenerator():
    # Geração das chaves privadas X448 e Ed448
    x448_private_key = X448PrivateKey.generate()
    ed448_private_key = Ed448PrivateKey.generate()

    # Geração das chaves públicas X448 e Ed448
    x448_public_key = x448_private_key.public_key()
    ed448_public_key = ed448_private_key.public_key()

    # Acordo de chaves com X448
    shared_key = x448_private_key.exchange(x448_public_key)

    # Perform key derivation.
    derived_key = HKDF(
        algorithm=hashes.SHA256(),
        length=32,
        salt=None,
        info=b'handshake data',
    ).derive(shared_key)

    # Assinatura da chave compartilhada com Ed448
    signature = ed448_private_key.sign(derived_key)

    # Verificação da assinatura com Ed448
    try:
        ed448_public_key.verify(signature, derived_key)
        print("A assinatura é válida.")
    except:
        print("A assinatura é inválida.")

    # Criptografar os dados
    nonce = os.urandom(12)  # 96 bits
    return derived_key, nonce

Implementação da comunicação cliente-servidor

* `handle_echo` - comporta-se como o server, onde está à escuta por um cliente e recebe uma mensagem encriptada e decifra-a.

* `tcp_echo_client` - como o nome refere, trata-se do cliente, onde este manda uma mensagem encriptada para o servidor e recebe de volta a sua mensagem original.

* `main` - função para pôr ambos a correr

Aqui o canal privado (a linha vermelha) pode ser visto como as variáveis globais, onde sabemos que é passada informação que não pode ser explorada.


In [6]:
# Server
async def handle_echo(reader, writer):
    data = await reader.read(100)
    message = data

    addr = writer.get_extra_info('peername')

    print(f"Server has received {message} from client {addr}")

    print("Decrypting...")
    data = receiver(key, nonce, associateddata, message)

    print(f"Server has sent: {data}")
    writer.write(data)
    await writer.drain()

    print("Closing the server connection")
    writer.close()
    

async def main():
    server = await asyncio.start_server(
        handle_echo, '127.0.0.1', 8890)

    addr = server.sockets[0].getsockname()
    print(f'Serving on {addr}')

# Client
async def tcp_echo_client():
    await asyncio.sleep(3)
    reader, writer = await asyncio.open_connection(
        '127.0.0.1', 8890)
    
    print("Sending: ", plaintext)

    print('Encrypting...')
    cifra = emitter(plaintext, key, nonce, associateddata)

    print(f'Client has sent: {cifra}')
    writer.write(cifra)

    data = await reader.read(100)
    print(f'Client has received: {data}')

    print('Closing the client connection')
    writer.close()

async def run_client_and_server():
    await asyncio.gather(tcp_echo_client(), main())


plaintext = b"Anacleto manda mensagem a Bernardina"
key, nonce = pseudoRandomGenerator()
chacha = ChaCha20Poly1305(key)
associateddata = b"ASCON"

await run_client_and_server()

A assinatura é válida.
Serving on ('127.0.0.1', 8890)
Sending:  b'Anacleto manda mensagem a Bernardina'
Encrypting...
Client has sent: b"\xe0\x7f\x9d\xc9'\xa6B\x97_FZ\xc5\tQ\xd8\x0f:nO\xb4\xe1\xe9F\xc1\xb8\xc3\x06\xf4n\x88sE\x19\xd0\x10)\x9f\xe4\xa5\x05\xb0K\xfa\r\x04b}iN}p\xea"
Server has received b"\xe0\x7f\x9d\xc9'\xa6B\x97_FZ\xc5\tQ\xd8\x0f:nO\xb4\xe1\xe9F\xc1\xb8\xc3\x06\xf4n\x88sE\x19\xd0\x10)\x9f\xe4\xa5\x05\xb0K\xfa\r\x04b}iN}p\xea" from client ('127.0.0.1', 53850)
Decrypting...
Server has sent: b'Anacleto manda mensagem a Bernardina'
Closing the server connection
Client has received: b'Anacleto manda mensagem a Bernardina'
Closing the client connection
